# Test compositi
### esempio A ([link](https://ntrs.nasa.gov/api/citations/19950009349/downloads/19950009349.pdf) pag 31 e 46)

laminato 4 ply [0,+45]s simmetrico con:

- E1 = 20010000 [lb/in^2]
- E2 = 1301000 
- G12 = 1001000
- v12 = 0.3 (v21 = 0.02)
- thk = 0.005

Determino per la ply a zero gradi:

- Q11 = 20130785
- Q12 = 392656
- Q22 = 130853
- Q16 = 0
- Q66 = 1001000

In [18]:
import numpy as np
from Composito import get_laminate,Q_ply,get_engineering_constants,get_compliance

def matprint(mat, fmt="g"): # https://gist.github.com/braingineer/d801735dac07ff3ac4d746e1f218ab75
    col_maxes = [max([len(("{:"+fmt+"}").format(x)) for x in col]) for col in mat.T]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:"+str(col_maxes[i])+fmt+"}").format(y), end="  ")
        print("")
        
ply_0 = {'s':0.005,'Ex':20010000,'Ey':1301000,'v':0.3,'G':1001000,'fabric':False,'theta':0}
ply_c = {'s':0.005,'Ex':20010000,'Ey':1301000,'v':0.3,'G':1001000,'fabric':False,'theta':45}
print(f" ** Noti Ex,Ey,v12 trovo v21 = {ply_0['v']*(ply_0['Ey']/ply_0['Ex'])}")
print(f" ** Q per la ply a 0 gradi \n{Q_ply(ply_0)}")

 ** Noti Ex,Ey,v12 trovo v21 = 0.019505247376311843
 ** Q per la ply a 0 gradi 
[[20127779.19369262   392597.31230876        0.        ]
 [  392597.31230876  1308657.70769586        0.        ]
 [       0.                0.          1001000.        ]]


Determino per la ply a 45 gradi:

- Q11 = 6557237
- Q12 = 4557238
- Q22 = 6557237
- Q16 = 4705483
- Q26 = 4705483
- Q66 = 5163582

In [19]:
print(f" ** Q per la ply a 45 gradi \n {Q_ply(ply_c)}")

 ** Q per la ply a 45 gradi 
 [[6556407.8815015  4554407.8815015  4704780.37149919]
 [4554407.8815015  6556407.8815015  4704780.37149919]
 [4704780.37149919 4704780.37149919 5162810.56919274]]


Determino le matrici del laminato

- A11 = 266880 
- A12 = 49479
- A22 = 78661
- A16 = 47055
- A26 = 47055
- A66 = 61646

### D

- D11 = 12.3 [inlb]
- D12 = 0.609
- D22 = 1.31
- D16 = 0.392
- D26 = 0.392
- D66 = 1.014

In [21]:
laminato = [ply_0,ply_c,ply_c,ply_0]
A,B,D,_ = get_laminate(laminato)
print('- A')
print(A)
print('- B')
print(B)
print('- D')
print(D)


cedevolezza = get_compliance(A,B,D)
print('- CEDEVOLEZZA TOTALE')

matprint(cedevolezza)


- A
[[266841.87075194  49470.0519381   47047.80371499]
 [ 49470.0519381   78650.65589197  47047.80371499]
 [ 47047.80371499  47047.80371499  61638.10569193]]
- B
[[ 0.00000000e+00 -3.55271368e-15  0.00000000e+00]
 [-3.55271368e-15 -7.10542736e-15  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  7.10542736e-15]]
- D
[[12.28757185  0.60854909  0.39206503]
 [ 0.60854909  1.30975099  0.39206503]
 [ 0.39206503  0.39206503  1.01415088]]
- CEDEVOLEZZA TOTALE
      266842       49470.1      47047.8             0  -3.55271e-15            0  
     49470.1       78650.7      47047.8  -3.55271e-15  -7.10543e-15            0  
     47047.8       47047.8      61638.1             0             0  7.10543e-15  
           0  -3.55271e-15            0       12.2876      0.608549     0.392065  
-3.55271e-15  -7.10543e-15            0      0.608549       1.30975     0.392065  
           0             0  7.10543e-15      0.392065      0.392065      1.01415  


Determino 
- Ex = S_x/e_x = (N_x/h)/e_x = 11333000
- Nx = A*e_x


In [22]:
Ex,Ey,vxy = get_engineering_constants(A,s = 4*ply_0['s'])
print(Ex)
Ex,Ey,vxy = get_engineering_constants(cedevolezza,s = 4*ply_0['s'])
print(Ex)
C,s = get_laminate(laminato,Compliance = True)
print('[Ex,Ey,v] = ...')
print(get_engineering_constants(C,s))


11331461.01058543
11331461.01058543
[Ex,Ey,v] = ...
(11331461.01058543, 2097169.9082667995, 0.31724502386941955)


### stress in laminate

Nx = 500 [lb/in]

In [23]:
A_i= np.linalg.inv(A)
D_i= np.linalg.inv(D)
B = np.zeros((3,3))
temp_sup = np.concatenate((A_i,B), axis =1)
temp_inf = np.concatenate((B,D_i), axis =1)
rigidezza = np.concatenate((temp_sup,temp_inf))
N = [500,0,0]
M = [0,0,0]
sollecitazioni = np.concatenate((N,M))
np.dot(rigidezza,sollecitazioni)
print('- rigidezza TOTALE')
matprint(rigidezza)
print(A_i[0,0]*500)

- rigidezza TOTALE
 4.41249e-06  -1.39984e-06  -2.29953e-06           0           0           0  
-1.39984e-06   2.38417e-05  -1.71296e-05           0           0           0  
-2.29953e-06  -1.71296e-05   3.10538e-05           0           0           0  
           0             0             0   0.0836422  -0.0330024  -0.0195771  
           0             0             0  -0.0330024    0.876445    -0.32607  
           0             0             0  -0.0195771    -0.32607     1.11967  
0.0022062468358357264


### esempio B ([link](https://ntrs.nasa.gov/api/citations/19950009349/downloads/19950009349.pdf) pag 38)

laminato 2 ply [0,+45] stesso di prima

Nota Aij, pari all'esempio B.

Determino per LAMINATO:

### B

- B11 = 170 [lb]
- B12 = -52
- B22 = -66
- B16 = -59
- B16 = -59
- B66 = -52

### D

- D11 = 1.112 [inlb]
- D12 = 0.206
- D22 = 0.328
- D16 = 0.196
- D26 = 0.196
- D66 = 0.257

Quindi le costnati ingegeristiche:

- Ex = 5839000

In [24]:
laminato = [ply_0,ply_c]
A,B,D,_ = get_laminate(laminato)
print('- A')
print(A)
print('- B')
print(B)
print('- D')
print(D)

temp_sup = np.concatenate((A,B), axis =1)
temp_inf = np.concatenate((B,D), axis =1)
cedevolezza = np.concatenate((temp_sup,temp_inf))
Ex,Ey,vxy = get_engineering_constants(cedevolezza,s = 2*ply_0['s'])
print(Ex)

- A
[[133420.93537597  24735.02596905  23523.9018575 ]
 [ 24735.02596905  39325.32794599  23523.9018575 ]
 [ 23523.9018575   23523.9018575   30819.05284596]]
- B
[[169.6421414  -52.02263211 -58.80975464]
 [-52.02263211 -65.59687717 -58.80975464]
 [-58.80975464 -58.80975464 -52.02263211]]
- D
[[1.11184113 0.20612522 0.19603252]
 [0.20612522 0.32771107 0.19603252]
 [0.19603252 0.19603252 0.25682544]]
5867147.696611008
